In [3]:
import motornet as mn

In [9]:
help(mn.effector)
effector = mn.effector.ReluPointMass24()
env = mn.environment.RandomTargetReach(effector=effector, max_ep_duration=1.)


Help on module motornet.effector in motornet:

NAME
    motornet.effector

CLASSES
    torch.nn.modules.module.Module(builtins.object)
        Effector
            ReluPointMass24
            RigidTendonArm26
                CompliantTendonArm26
    
    class CompliantTendonArm26(RigidTendonArm26)
     |  CompliantTendonArm26(timestep=0.0002, skeleton=None, muscle_kwargs: dict = {}, **kwargs)
     |  
     |  This is the compliant-tendon version of the :class:`RigidTendonArm26` class. Note that the default integration
     |  method is Runge-Kutta 4, instead of Euler.
     |  
     |  Args:
     |    timestep: `Float`, size of a single timestep (in sec).
     |    skeleton: A :class:`motornet.skeleton.Skeleton` object class or subclass. This defines the type of
     |      skeleton that the muscles will wrap around. If no skeleton is passed, this will default to the skeleton
     |      used in the parent :class:`RigidTendonArm26` class.
     |    
     |    **kwargs: All contents are